# Datenaufbereitung und -visualisierung mit modernen Python-Tools

**Dr. Martin Felder, ZSW, im Oktober 2022**

Python hat sich im Bereich Data Science als eine der erfolgreichsten Programmiersprachen durchgesetzt. Dies ist unter anderem der Entwicklung von immer ausgefeilteren Tools in den letzten Jahren zu verdanken, da die Sprache selber keine Arrays oder vektorielle Verarbeitung unterstützt.

Das erste Tool und damit die Basis für (fast) alle darauffolgenden ist das Paket `numpy`. Es ermöglicht u.a. die Verarbeitung von Arrays mit definierten Datentypen, sowie das einfache Lesen und Schreiben binärer Daten. Wir werden in diesem Tutorial allerdings nicht näher auf `numpy` eingehen, es wird aber von den besprochenen Paketen implizit verwendet.

Ein wichtiger Unterschied von `numpy` zu "modernen" Tools ist, dass Letztere bestimmte Metadaten mitverarbeiten und -abspeichern. Dies erlaubt eine viel komfortablere Filterung und Visualisierung von Daten. Außerdem vermeidet man viele Fehler, die ansonsten durch die unvermeidliche Manipulation von verschachtelten Indizes auftreten können.

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from datetime import datetime
import gzip

import holoviews as hv
import hvplot.pandas
import hvplot.xarray
import matplotlib.pyplot as plt

# Die folgenden drei Abkürzungen haben sich in der Community etabliert:
import numpy as np
import pandas as pd
import pandas_profiling
import xarray as xr

## Tabellenartige Daten - `pandas`

`pandas` erlaubt die Verarbeitung "tabellenartiger" Daten als `DataFrame`. Ein Beispiel zum Einlesen solcher Daten aus einem komprimierten CSV-File:

In [ ]:
with gzip.open('temperatur.csv.gz', 'rb') as f:
    df = pd.read_csv(f, index_col="time", parse_dates=True)
df.head()

Hier sieht man bereits die mitverarbeiteten Metadaten:

In [ ]:
df.columns  # Eine Liste von Labels für die Spalten

In [ ]:
df.index  # Eine Liste von Labels für die Zeilen

Diese Attribute lassen sich auch mit beliebigen Iterables überschreiben (sofern die Dimensionen stimmen). Die eigentlichen Daten liegen in Form eines `numpy` Arrays unter `df.values` vor, welches logischerweise die Dimension (# Zeilen, # Spalten) hat. Im Wesentlichen sind die Zeilen- und Spaltenindizes gleichberechtigt, man kann sie auch leicht durch Transposition vertauschen:

In [ ]:
df.T.index

Man kann den `DataFrame` aber auch als eine Art `dictionary` von Variablen ansehen, welches mit den Spaltennamen indiziert ist (die sich aus Conveniencegründen auch als Attribute schreiben lassen). Dies hat insofern seine Berechtigung, als die `dtypes` **pro Spalte** festgelegt sind:

In [ ]:
df["name"].dtype, df.T02.dtype

Darauf kommen wir später zurück. Spalten lassen sich natürlich entfernen oder umbenennen, oder wie bei einem `dictionary` hinzufügen (siehe Anleitung für Details).

In [ ]:
df.drop(columns=["Einstrahlung", "name"]).rename(columns={"T12": "T13"}).head()

Mit den Attributen `.iloc` und `.loc` kann auf Subsets des `DataFrame`s zugegriffen werden, wobei `.iloc` einen numerischen Index erwartet und `.loc` einen, der die Labels von Zeilen und Spalten enthält:

In [ ]:
df.iloc[0:2, 0:3]

Die Indizierung erfolgt hier also wie bei `numpy`. Allerdings ist das Ergebnis wieder ein `DataFrame` - das gleiche Subset kann man sich natürlich auch als Array zurückgeben lassen:

In [ ]:
df.values[0:2, 0:3]

Die Indizierung mit Labels ist sehr mächtig und umfasst u.a. Typkonversionen usw. Dies umfassend darzustellen übersteigt den Rahmen des Tutorials und wird hiermit der [Anleitung](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#selection-by-label) überlassen. Ein einfaches Beispiel:

In [ ]:
df.loc["2022-06-29":"2022-06-29 00:20:00", "T01":"T04"]

Durch die Mächtigkeit entstehen leider gelegentlich unintuitive Konventionen und/oder unerwartetes Verhalten. Bei Problemen bitte genau in der Anleitung nachlesen oder googlen. 

### Einlesen bzw. Erzeugen von `DataFrame`s

DataFrames lassen sich wie oben gezeigt mittels `pd.read_csv` aus CSV-Dateien einlesen. Der Parser ist sehr mächtig und macht es in den meisten Fällen unnötig, an den Dateien selber etwas zu verändern (im Gegensatz zum Excel-Parser...). Die nach meiner Erfahrung nützlichsten Argumente sind:

* `index_col`: Name oder laufende Nummer (0-based) der Spalte, die als Index verwendet werden soll.
* `parse_dates`: `True`, wenn der Index als Zeit interpretiert werden soll. Man kann auch mehrere Spalten angeben, die [zusammen interpretiert werden](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html#datetime-handling).
* `dayfirst`: Wichtig für europäische Datumsformate!
* `delimiter`: Das Spaltentrennzeichen.
* `skiprows`: Erlaubt das Überspringen von Headerzeilen am Anfang der Datei.
* `header`: Spezifiziert, welche Zeile als Spaltennamen verwendet werden soll (oder `False`).
* `encoding`: Falls die Datei Umlaute o.ä. enthält und unter Windows oder auf älteren Macs erzeugt wurde, muss hier wahrscheinlich entweder `'latin1'`, `'iso-8859-1'`, `'cp1252'` oder `'utf-16'` stehen... 
* `decimal`: Dezimaltrennzeichen, default `"."`. Für deutsche Zahlenformate `","` einstellen.


Pandas kann mittels `pd.read_excel` auch direkt Excel-Tabellenblätter einlesen. Dies hat den Vorteil, dass Komplikationen wie das Encoding oder die Dezimaltrennzeichen wegfallen. Allerdings kann es Probleme geben, wenn die Zellentypen schon im Excel falsch eingestellt sind, z.B. Zahlen als Text. Ansonsten ist der [Parser](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html#reading-excel-files) relativ ähnlich zum CSV-Parser.

In [ ]:
sot = pd.read_excel("solarthermie.xlsx")
sot.head()

In diesem Beispiel liegt der Zeitstempel auf zwei Spalten verteilt vor. Da es sich aber nicht im Strings handelt, können sie nicht wie bei `parse_dates` zusammen interpretiert werden. Einen entsprechenden Zeitindex erhält man aber wie folgt:

In [ ]:
sot = sot.set_index(sot.apply(lambda row: datetime.combine(row.date, row.time), 1))
sot.head()

Hier fällt auf, dass die Zeitstempel keine Zeitzone enthalten und der Index keinen Namen hat (oben hieß er `time`).

#### Ärger mit der Lokalzeit

Aber was passiert, wenn die ursprünglichen Daten in Lokalzeit abgespeichert sind, dies aber aus dem Zeitstempel-String nicht ersichtlich ist?

In [ ]:
lz = pd.read_csv(
    "solarthermie_lokalzeit.csv",
    index_col="timestamp",
    parse_dates=True,
    delimiter=";",
    decimal=",",
)
lz

Hier sind die Sprünge am Übergang zwischen Sommer- und Winterzeit leicht zu erkennen. **Dies bitte unbedingt überprüfen, wenn die Daten nicht eindeutig in UTC vorliegen!** Falsch interpretierte Zeitstempel sind Teufelswerk!

Wenn die Zeitzone, in der die Daten vorliegen, bekannt ist, kann der Index mit `.tz_localize` als diese Zeitzone uminterpretiert werden.

In [ ]:
lz = lz.tz_localize("Europe/Berlin", ambiguous="infer")
lz

Es ist aber auf jeden Fall empfehlenswert **alles in UTC zu konvertieren und die Zeitzone dann zu entfernen**. 

In [ ]:
lz.tz_convert("UTC").tz_localize(None)

Wer häufig Probleme mit Zeitstempeln und Datumsberechnungen hat, sollte sich das speziell dafür optimierte Paket `arrow` ansehen.

In [ ]:
df = df.drop(columns="Einstrahlung").tz_localize(None)  # Dataframe aufräumen

### (Statistische) Beschreibung von `DataFrame`s

Für eine erste Übersicht reicht es meist, die Datentypen und Vollständigkeit der einzelnen Spalten zu kennen. Dazu dient:

In [ ]:
sot.info()

Hier bietet es sich bereits an, die offenbar leeren Spalten zu entfernen, sowie die unvollständigen Zeilen (außer diese sollen interpoliert werden o.ä.). Wir zeigen hier nur den Befehl, weisen aber den Output keiner Variable zu.

In [ ]:
sot.drop(columns="Energie").dropna()

Eine etwas ausführlichere statistische Beschreibung erhält man mit 

In [ ]:
sot.describe()

Die Stabilität der Mittelwerte aller Daten lässt sich z.B. mit dem ebenfalls implementierten Bootstrap-Plot untersuchen.

In [ ]:
%matplotlib inline
pd.plotting.bootstrap_plot(sot.TemperaturVL, fig=None, size=100, samples=1000)

Wer es noch genauer wissen möchte, kann sich über das `pandas-profiling` Paket einen kompletten Statistikreport ausgeben lassen:

In [ ]:
sot.profile_report()

### Selektion, Manipulation und Visualisierung

Die Möglichkeiten zur Manipulation von DataFrames könnte man als eine Kombination von Excel- und SQL-Operationen ansehen, auch von der Mächtigkeit her. Wir kratzen hier kurz an der Oberfläche und sehen uns einige häufig verwendete Methoden an.

Wichtig ist zunächst die Selektion der gewünschten Daten. Abgesehen von der Adressierung nach Labels und Integer-Indizes können natürlich auch Datenbereiche ausgewählt werden. Dazu gibt es verschiedene Methoden, die sich i.W. in der Performance unterscheiden:

In [ ]:
%timeit sot.iloc[(sot.DeltaTemp.values > 19) & (sot.Durchfluss.values > 2.7), :]

In [ ]:
%timeit sot[(sot.DeltaTemp > 19) & (sot.Durchfluss > 2.7)]

In [ ]:
%timeit sot.query("DeltaTemp > 19 & Durchfluss > 2.7")

In [ ]:
%timeit sot.where((sot.DeltaTemp > 19) & (sot.Durchfluss > 2.7))

Beim Operator `in` ist zu beachten, dass dieser sich nicht Index oder Daten bezieht, sondern auf die Spalten. Der DataFrame verhält sich in diesem Fall wie ein `dictionary` der Spalten, wie oben:

In [ ]:
"sungo" in df, pd.Timestamp("2022-06-28 05:28:45") in df, "T01" in df

Ob gewisse Werte in den Daten enthalten sind, findet mal mit `.isin` heraus:

In [ ]:
df.isin(["sungo"]).head()

Downsampling von Zeitreihen ist mit Pandas kein Problem (zum Upsampling kommen wir weiter unten).

In [ ]:
df.resample("h").mean().head()

Pandas enthält grundlegende Plotting-Funktionalität auf der Basis von Matplotlib. Das Statistikplot-Package `seaborn` ist ebenfalls mit Pandas kompatibel, wird aber aus Zeitgründen hier nicht behandelt. Einfache Plots erhält man aus einzelnen Spalten, oder unter Verwendung der Spaltennamen direkt aus dem ganzen `DataFrame`.

In [ ]:
%matplotlib inline
df.T01.plot(figsize=(20, 5), grid=True, label="T01", legend=True)
df.T03.plot(label="T03", legend=True);

In [ ]:
df.plot.scatter(x="T01", y="T03", c="T06", alpha=0.3, cmap="rainbow", figsize=(8, 6));

Eine weniger bekannte Alternative ist das Paket `hvplot`, das ein auf den ersten Blick sehr ähnliches Interface bietet, aber auf `bokeh` bzw. `holoviews` aufsetzt.

In [ ]:
df.hvplot.scatter(
    x="T01",
    y="T03",
    c="T06",
    alpha=0.3,
    cmap="rainbow",
    frame_width=500,
    aspect=1,
    clabel="T06 [°C]",
)

In [ ]:
df.T01.hvplot(width=1200, grid=True)

Natürlich sind beide Lösungen nicht perfekt und haben jeweils ihre Macken. In diesem Tutorial werden wir uns auf `hvplot` konzentrieren. Wie bei allen high-level Plottingtools können manche Probleme nur auf den darunterliegenden Ebenen gelöst werden. Bei `hvplot` sind dies wie gesagt zunächst `holoviews`, noch weiter darunter liegt die `bokeh`-Schicht (die auf `D3.js` aufsetzt). Gewisse Hacks lassen sich oft leicht ergooglen, wie z.B.

In [ ]:
# Zeitachse auf deutsches Format unstellen
from bokeh.models.formatters import DatetimeTickFormatter

datefmt = DatetimeTickFormatter(
    hours="%d.%m. %H:%M", days="%d.%m.", months="%Y-%m", years="%Y-%m"
)

In [ ]:
df.T01.hvplot(width=1200, grid=True, xformatter=datefmt)

Ob man ein high-level Tool verwendet oder lieber z.B. auf der Matplotlib-Ebene bleibt, ist letztlich Geschmackssache. Nach meiner persönlichen Erfahrung sparen high-level Plots für schnelle Datensichtungen und -auswertungen nach einer gewissen Einarbeitung durchaus Zeit, auch wenn man gelegentlich an die Grenzen stößt und etwas Zeit auf der Suche nach speziellen Lösungen verliert.

`hvplot` erzeugt `holoviews`-Objekte, die sich nach einem [ausgeklügelten Schema](https://holoviews.org/user_guide/Building_Composite_Objects.html) zusammensetzen lassen. Wir kratzen hier auch nur an der Oberfläche.

In [ ]:
opts = dict(width=1200, grid=True, xformatter=datefmt)
plot1 = df.T01.hvplot(**opts) 
plot2 = df.T03.hvplot(**opts)
(plot1 + plot2).cols(1)

In [ ]:
plot1 * plot2

Je größer die Daten sind, desto nützlicher wird die Visualisierung in zwei Dimensionen, z.B. als Heatmap.

In [ ]:
df.groupby(df.index.hour).mean().hvplot.heatmap(ylabel="hour of day", cmap="RdYlBu_r")

In [ ]:
(df.T01 - df.T06).rename("diff").hvplot.heatmap(
    x="time.day_of_year",
    y="time.hour",
    C="diff",
    cmap="RdBu_r",
    symmetric=True,
    clabel="T01 – T06 [°C]",
).aggregate(function=np.mean)

In [ ]:
(df.T01 - df.T06).rename("diff").hvplot.heatmap(
    x="time.day_of_year",
    y="time.hour",
    C="diff",
    cmap="reds",
    clabel="RMSD (T01 – T06) [°C]",
).aggregate(function=lambda x: np.sqrt(np.mean(np.square(x))))

Die Daten der beiden DataFrames sollen nun zusammengefügt werden. Die Zeitstempel scheinen in etwa im 3min-Raster aufgenommen worden zu sein. Wir überprüfen das wie folgt:

In [ ]:
plots = [
    pd.DataFrame(np.diff(mydf.index.values.astype(np.int64)) / 1e9)
    .clip(150, 200)
    .hvplot.hist(bins=100, xlabel="Messintervall [s]")
    for mydf in [df, sot]
]
hv.Layout(plots)

Hierbei wurde ausgenutzt, dass die Zeitstempel (im Kern `numpy.datetime64`) intern als Nanosekunden ab "Epochenbeginn" vorliegen. Sie lassen sich also leicht in Integerwerte umwandeln, mit denen dann gerechnet werden kann. **Bei größeren Datumsmanipulation in Pandas kann das deutlich schneller sein als mit `pd.Timestamp`.**

In [ ]:
print(sot.index[0], type(sot.index[0]))
print(sot.index.values[0], type(sot.index.values[0]))
sec = sot.index.values[0].astype("i8") / 1e9
print(sec, "Sekunden")
print(pd.Timestamp(sec * 1e9))
print(sec.astype("M8[s]"))  # mit [s] als sekundengenau interpretiert

Wir möchten nun alle Daten auf einen gemeinsamen Zeitindex mit 3min-Intervall interpolieren. Hier stößt Pandas aufgrund seiner Herkunft (aus den Wirtschaftswissenschaften!) an die Grenzen: `.interpolate` füllt nur bestehende NaNs auf, `.resample("3min").mean().interpolate()` ist nicht korrekt, weil dann in manchen Intervallen zwei Werte gemittelt werden, in manchen gar keine. Das kann man überprüfen mit

In [ ]:
df.resample("3min").mean().isna().sum(axis=0)

Erzeugen wir zunächst einen neuen, gemeinsamen Zeitindex mit

In [ ]:
newidx = pd.date_range(
    max(df.index[0], sot.index[0]).floor("h"),
    min(df.index[-1], sot.index[-1]),
    freq="3min",
)
newidx

Dann können wir den ursprünglichen DataFrame mit den neuen Indices ergänzen, die enstehenden NaNs interpolieren und anschließend alles wieder auf den neuen Index einschränken:

In [ ]:
pd.concat(
    [df.drop(columns="name"), pd.DataFrame(0, index=newidx, columns=["_dummy"])], axis=1
).interpolate().drop(columns="_dummy").reindex(newidx).dropna().head()

O weh! Das muss doch irgendwie einfacher gehen??

## Mehrdimensionale Daten - `xarray`

`xarray` setzt auf Pandas auf und ermöglicht eine "naturwissenschaftlichere" Betrachtungsweise der Daten als ein- oder mehrdimensionale `DataArray`s, deren Dimensionen mit **Koordinatenachsen** versehen sind. Ein Pandas `DataFrame` kann auf natürliche Weise in eine Kollektion von `DataArray`s überführt werden. Das enstehende `Dataset` verhält sich bezüglich der Variablen wieder in etwa wie ein `dict`:

In [ ]:
xa = (
    sot.drop(columns=["time", "date", "name", "anlage"])
    .to_xarray()
    .rename(index="time")
)
xa

Als Metadaten hat man hier:

+ `dims`: Dimensionsnamen für jede Koordinatenachse. 
+ `coords`: Ein `dict`-ähnlicher Container von (typischerweise) 1-dimensionalen Koordinatenvektoren für die einzelnen Dimensionen.
+ `attrs`: Ein `OrderedDict` mit beliebigen Metadaten. Bestimmte standardisierte Keys (z.B. `"units"`) werden direkt beim Plotten verwendet.

Einer der Vorteile von `xarray` ist dadurch, dass der Zwang aufgehoben wird, alle Metainformationen über eine Variable in den Spaltennamen zu pressen – der möglichst auch noch Python-konform sein soll, damit man ihn über ein Attribut referenzieren kann. Ein Beispiel:

In [ ]:
xa.time.attrs["long_name"] = "Messzeitpunkt"
xa.TemperaturVL.attrs["long_name"] = "Vorlauftemperatur"
xa.TemperaturVL.attrs["units"] = "°C"
t = xa.TemperaturVL
t

Bei Bedarf können natürlich noch andere Informationen wie Messpunkt-ID, Genauigkeit usw. als Attribute gesetzt werden. Damit ist diese Messgröße vollständig beschrieben und bedarf keiner externen Dokumentation mehr!

Das Plotten erfolgt ganz analog zu Pandas. Man beachte die automatische Verwendung der Metadaten. 

In [ ]:
t[:100].hvplot(label="original")

Lineare Interpolation von Daten ist nun mit einem einzelnen Methodenaufruf möglich.

In [ ]:
t.hvplot(label="original") * t.interp(time=newidx).hvplot(label="interpoliert")

Mit der Methode `xr.combine_by_coordinates` lassen sich verschiedene `Datasets` automatische zusammenfügen, auch in mehreren Dimensionen gleichzeitig:

In [ ]:
xr.combine_by_coords(
    [xa.interp(time=newidx), df.drop(columns="name").to_xarray()], data_vars="all"
)

In [ ]:
wp = xr.load_dataset("wind_fc.nc")
wp

In [ ]:
wp1 = wp.isel(time=slice(None, 100), fc=slice(100, None))
wp2 = wp.isel(time=slice(150, 200), fc=slice(0, 50))
xr.combine_by_coords([wp2, wp1])

In [ ]:
wp.targ.hvplot.image() + xr.combine_by_coords([wp2, wp1]).targ.hvplot.image()

Aggregation erfolgt ebenfalls analog zu Pandas. Es können auch mehrere Dimensionen angegeben werden.

In [ ]:
wp.mean(dim="time").hvplot()

Werden für einen Plot nicht alle Dimensionen verwendet, erzeugt `hvplot` (normalerweise) ein Widget, mit dem die fehlenden Dimensionen interaktiv eingestellt werden können.

In [ ]:
wp.hvplot(x="fc", ylabel=f"wind speed [m/s]")

Eine praktische Option für divergierende Colormaps ist `symmetric`:

In [ ]:
myplot = (
    (wp.pred - wp.targ)
    .rename("wind speed bias")
    .hvplot.heatmap(
        x="time.day",
        y="time.hour",
        C="wind speed bias",
        cmap="RdBu_r",
        symmetric=True,
        clabel="m/s",
    )
)
myplot

### Hilfskoordinaten

Nehmen wir an, wir möchten den Fehler der Vorhersage zu jeder Tageszeit bestimmen. Da `time` nur eindimensional ist, können wir sie nicht verwenden, um jeder Stunde einen Fehler zuzuweisen. Wir bauen daher eine zweidimensionale Hilfskoordinate.

Da wir für jeden Vorhersagewert eine `hr_of_day` benötigen, zählen wir erst die Vorhersagestunde hinzu und nehmen das ganze dann Modulo 24. Das funktioniert, wenn der erste Zeitstempel um 00:00 ist. 

In [ ]:
hr_since_begin = (wp.time - wp.time[0]).astype(
    int
) / 3.6e12  # ns -> h
dtime = sum(np.meshgrid(wp.fc, hr_since_begin))

Das Ergebnis benutzen wir als 2D-Hilfskoordinate und erstellen auch noch eine Tageskoordinate, 
damit wir ganz einfach Statistik betreiben können.

In [ ]:
wp = wp.assign_coords(
    hr_of_day=(("time", "fc"), dtime.astype(int) % 24),
    day=(("time",), wp.time.dt.floor("D").data),
    hr_since_begin=(("time",), hr_since_begin.data),
)
wp

In [ ]:
(wp.pred - wp.targ).groupby("hr_of_day").apply(
    lambda x: np.sqrt(np.mean(np.square(x)))
).rename("RMSE [m/s]").hvplot()

## Abspeichern hochaufgelöster Plots

Typischerweise spielt `hvplot` seine Stärken mehr in einer interaktiven Umgebung aus. Anbei der Vollständigkeit halber noch ein Trick, um `holoviews`-Plots (keine Layouts!) als SVG und PNG in hoher Auflösung zu exportieren.

In [ ]:
# some hack to export SVG
from subprocess import run

from bokeh.io import export_svg as bokeh_export


def savepic(obj, filename, dpi=200):
    plot_state = hv.renderer("bokeh").get_plot(obj).state
    plot_state.output_backend = "svg"
    filepath = f"{filename}.svg"
    bokeh_export(plot_state, filename=filepath)
    run(
        f"convert -density {dpi} {filepath} {filepath.replace('.svg', '.png')}",
        shell=True,
    )
    return obj

In [ ]:
#savepic(myplot, "testplot")  # funktioniert hier nicht, wegen fehlender Pakete

## Effiziente Datenformate

Für größere, numerische Datenmengen empfielt es sich nicht, alles in CSV-Dateien abzuspeichern, da derartige ASCII-Formate sehr ineffizient sind. Moderne binäre Datenformate sind durch **chunking**, **lazy loading** und **distributed access** häufig um ein Vielfaches schneller.

1. **chunking** bedeutet, dass die Daten nicht als einzelne Datei auf der Festplatte liegen, sondern als intelligent aufgeteilte Blöcke. Die Laderoutinen erlauben dann selektives Laden eines Teils der Daten, bei dem nur die benötigten Blöcke tatsächlich gelesen werden.
2. Beim **lazy loading** werden beim Aufrufen der Laderoutine nur die Metadaten eingelesen. Der Rest folgt erst, wenn tatsächlich auf die Daten zugegriffen wird.
3. **distributed access** meint das gleichzeitige Laden mehrerer Blöcke in verschiedenen Prozessen oder Threads, mit für den User transparenter Zusammenführung im Speicher.

### Pandas

**`.to_hdf`** erlaubt das Speichern von `DataFrames` im HDF5-Format mittels des `pytables`-Pakets. Dazu sollte die Option `pd.set_option('io.hdf.default_format','table')` gesetzt werden; das Defaultformat `fixed` ist eher veraltet. HDF5 ist auch nicht wirklich "modern" im Sinne der oben beschriebenen Kriterien, aber es unterstützt Kompression, mehrere `DataFrames` in einer Datei sowie das Anhängen von Daten an bestehende Files.

**`.to_parquet`** ist eine modernere und effizientere Alternative, die zumindest die Kriterien 1 und 2 erfüllt. Parquet nutzt spaltenweise, schnelle Komprimierungsalgorithmen. Das Anfügen an bestehende Daten wird nicht nativ von `.to_parquet` unterstützt, ist aber mit einem kleinen Hack möglich.

### xarray

**`.to_netcdf`** ist der Standard für `xarray`. Es wird ein netCDF4-File (setzt auf HDF5 auf) geschrieben, das bei der Wahl entprechender Attribute dem [CF-Conventions-Standard](https://cfconventions.org/) entspricht. NetCDF4 selber erfüllt die obigen Kriterien nicht, aber `xarray` erlaubt sozusagen "manuelles" chunking, indem die Routine `.open_mfdataset` eine beliebige Liste von Files gleichzeitig öffnen und anhand der enthaltenen Dimensionen in ein `Dataset` kombinieren kann. In Kombination mit dem `dask`-Paket lassen sich so Pipelines bauen, bei denen nie die ganzen Daten gleichzeitig im Speicher sind. 

**`.to_zarr`** ist ein neueres Feature und bietet automatisches chunking sowie die Möglichkeit, die Daten direkt in diversen Cloud-Speichern abzulagen. Auch ansonsten ist das Format Parquet sehr ähnlich, erlaubt aber auch das Anhängen von Daten entlang einer Dimension. Ein Beispiel:


In [ ]:
filename = f"temp{np.random.randint(999999):0d}"
ds0 = xr.Dataset(
    {"temperature": (["time"], [50, 51, 52])},
    coords={"time": pd.date_range("2000-01-01", periods=3)},
)
ds1 = xr.Dataset(
    {"temperature": (["time"], [53, 54, 55])},
    coords={"time": pd.date_range("2000-01-04", periods=3)},
)

ds0.to_zarr(filename)
ds1.to_zarr(filename, mode="a", append_dim="time")

ds2 = xr.open_zarr(filename)

In [ ]:
ds2